<a href="https://colab.research.google.com/github/codingle2/Data_Analysis_Programming_Class/blob/main/crawling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 필요 라이브러리 설치 (Colab에는 대부분 설치되어 있으나 확인용)
# !pip install requests pandas beautifulsoup4 openpyxl

import requests
import json
import time
import re
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime

# ==========================================
# 1. 유틸리티 함수 (데이터 변환 등)
# ==========================================

def parse_krw_hangeul(value):
    """
    거래대금(억) 단위의 한글 표현을 정수로 변환
    예 : "1,234억원" -> 123400000000
    """
    if value is None:
        return 0
    value = str(value).replace(',', '').replace(' ', '')
    if value.endswith('억원'):
        try:
            num = float(value.replace('억원', ''))
            return int(num * 100000000)
        except ValueError:
            return 0
    try:
        return int(value)
    except ValueError:
        return 0

# ==========================================
# 2. 메인 크롤링 함수들
# ==========================================

def get_top_stocks_data():
    """
    네이버 금융 API에서 거래량 상위 주식 데이터를 가져옵니다.
    DB 저장 대신 DataFrame으로 반환합니다.
    """
    stocks_per_page = 20
    base_api_url = "https://m.stock.naver.com/api/stocks/quantTop/all"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
        "Accept": "application/json, text/plain, */*",
        "Referer": "https://m.stock.naver.com/"
    }

    total_api_pages_to_fetch = 2  # 테스트를 위해 2페이지(40개)만 가져옵니다. 필요시 늘리세요.
    fetched_data = []

    print(f"=== 상위 주식 데이터 크롤링 시작 (총 {total_api_pages_to_fetch * stocks_per_page}개 목표) ===")

    for i in range(1, total_api_pages_to_fetch + 1):
        full_url = f"{base_api_url}?page={i}&pageSize={stocks_per_page}"
        print(f" -> 페이지 {i} 요청 중...")

        try:
            response = requests.get(full_url, headers=headers, timeout=5)
            response.raise_for_status()
            data = response.json()

            if isinstance(data, dict) and 'stocks' in data and isinstance(data['stocks'], list):
                fetched_data.extend(data['stocks'])
            else:
                print(f" -> 페이지 {i} 데이터 없음 혹은 형식 오류.")
                break

        except Exception as e:
            print(f" -> 페이지 {i} 오류 발생: {e}")
            break

        time.sleep(0.1) # 서버 부하 방지

    # 데이터 프레임으로 변환하여 보기 좋게 정리
    if fetched_data:
        df = pd.DataFrame(fetched_data)
        # 필요한 컬럼만 선택 및 이름 변경
        cols = {
            'itemCode': '종목코드',
            'stockName': '종목명',
            'closePrice': '현재가',
            'fluctuationsRatio': '등락률',
            'accumulatedTradingVolume': '거래량',
            'accumulatedTradingValueKrwHangeul': '거래대금'
        }
        # 존재하는 컬럼만 선택
        available_cols = [c for c in cols.keys() if c in df.columns]
        df = df[available_cols].rename(columns=cols)
        return df
    else:
        return pd.DataFrame()

def get_kospi_info():
    """네이버 KOSPI 기본 및 실시간 정보를 가져옵니다."""
    basic_url = "https://m.stock.naver.com/api/index/KOSPI/basic"
    realtime_url = "https://polling.finance.naver.com/api/realtime/domestic/index/KOSPI"

    result = {}

    # 기본 정보
    try:
        res = requests.get(basic_url, timeout=5).json()
        result['basic'] = res
    except Exception as e:
        print(f"KOSPI 기본 정보 오류: {e}")
        result['basic'] = None

    # 실시간 정보
    try:
        res = requests.get(realtime_url, timeout=5).json()
        result['realtime'] = res.get('result', res)
    except Exception as e:
        print(f"KOSPI 실시간 정보 오류: {e}")
        result['realtime'] = None

    return result

def get_stock_code_by_search(stock_name):
    """
    네이버 금융 검색을 통해 종목 코드를 가져옵니다 (BeautifulSoup 사용).
    """
    # 6자리 숫자인 경우 바로 반환
    if stock_name.isdigit() and len(stock_name) == 6:
        return stock_name

    search_url = f"https://finance.naver.com/search/search.naver?query={stock_name}"
    headers = {"User-Agent": "Mozilla/5.0"}

    try:
        response = requests.get(search_url, headers=headers, timeout=5)
        soup = BeautifulSoup(response.text, 'html.parser')

        # 종목 코드가 들어 있는 링크 찾기
        a_tags = soup.select('a')
        for a in a_tags:
            href = a.get('href', '')
            if '/item/main.naver?code=' in href:
                match = re.search(r'code=(\d{6})', href)
                if match:
                    return match.group(1)
    except Exception as e:
        print(f"종목 코드 검색 오류: {e}")

    return None

def get_stock_detail_info(code):
    """특정 종목의 상세 정보(현재가, 시가, 고가 등)를 크롤링합니다."""
    url = f"https://finance.naver.com/item/main.naver?code={code}"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
    }

    try:
        response = requests.get(url, headers=headers, timeout=5)
        soup = BeautifulSoup(response.text, 'html.parser')

        # 1. 주식명
        stock_name_tag = soup.select_one("div.wrap_company h2 a")
        stock_name = stock_name_tag.text.strip() if stock_name_tag else "알 수 없음"

        # 2. 현재가
        current_price_tag = soup.select_one("p.no_today span.blind")
        current_price = current_price_tag.text.strip().replace(",", "") if current_price_tag else "N/A"

        # 3. 기타 정보
        high_price = soup.select_one("table.no_info td:nth-of-type(2) span.blind")
        low_price = soup.select_one("table.no_info tr:nth-of-type(2) td:nth-of-type(2) span.blind")
        trade_volume = soup.select_one("table.no_info td:nth-of-type(3) span.blind")

        return {
            "종목코드": code,
            "종목명": stock_name,
            "현재가": current_price,
            "고가": high_price.text.strip().replace(",", "") if high_price else "N/A",
            "저가": low_price.text.strip().replace(",", "") if low_price else "N/A",
            "거래량": trade_volume.text.strip().replace(",", "") if trade_volume else "N/A",
        }

    except Exception as e:
        print(f"상세 정보 크롤링 오류: {e}")
        return {}

def get_daily_stock_prices(code, pages=3):
    """
    특정 종목의 일별 시세를 가져옵니다.
    pages: 가져올 페이지 수 (1페이지당 10일치)
    """
    prices = []
    headers = {"User-Agent": "Mozilla/5.0"}

    print(f"=== [{code}] 일별 시세 크롤링 시작 ({pages}페이지) ===")

    for page in range(1, pages + 1):
        url = f"https://finance.naver.com/item/sise_day.nhn?code={code}&page={page}"
        try:
            response = requests.get(url, headers=headers, timeout=5)
            soup = BeautifulSoup(response.text, 'html.parser')

            table = soup.find('table', class_='type2')
            if table:
                rows = table.find_all('tr')
                for row in rows:
                    cols = row.find_all('td')
                    # 유효한 데이터 행인지 확인 (날짜 포함 등)
                    if len(cols) >= 7:
                        date_text = cols[0].text.strip()
                        if date_text.count('.') == 2: # 날짜 형식 확인 (yyyy.mm.dd)
                            data = {
                                '날짜': date_text,
                                '종가': cols[1].text.strip().replace(',', ''),
                                '전일비': cols[2].text.strip().replace(',', '').replace('\t', '').replace('\n', ''),
                                '시가': cols[3].text.strip().replace(',', ''),
                                '고가': cols[4].text.strip().replace(',', ''),
                                '저가': cols[5].text.strip().replace(',', ''),
                                '거래량': cols[6].text.strip().replace(',', '')
                            }
                            prices.append(data)
            else:
                break
        except Exception as e:
            print(f"일별 시세 오류 (page {page}): {e}")
            break

    if prices:
        return pd.DataFrame(prices)
    else:
        return pd.DataFrame()

# ==========================================
# 3. 실행 예시 (Colab에서 바로 확인 가능)
# ==========================================

# 1) 상위 주식 정보 가져오기
print("\n[1] 거래량 상위 주식 목록")
df_top = get_top_stocks_data()
if not df_top.empty:
    print(df_top.head(5)) # 상위 5개만 출력
else:
    print("데이터를 가져오지 못했습니다.")

# 2) KOSPI 정보 가져오기
print("\n[2] KOSPI 정보")
kospi_data = get_kospi_info()
print("Basic:", kospi_data.get('basic', {}))
# print("Realtime:", kospi_data.get('realtime', {})) # 실시간 데이터는 양이 많을 수 있음

# 3) 종목 검색 및 상세 정보
target_stock_name = "삼성전자" # 검색할 종목명
print(f"\n[3] '{target_stock_name}' 검색 및 상세 정보")
stock_code = get_stock_code_by_search(target_stock_name)

if stock_code:
    print(f" -> 찾은 코드: {stock_code}")
    detail = get_stock_detail_info(stock_code)
    print(" -> 상세 정보:", detail)

    # 4) 일별 시세 가져오기
    print(f"\n[4] '{target_stock_name}'({stock_code}) 일별 시세")
    df_daily = get_daily_stock_prices(stock_code, pages=2)
    if not df_daily.empty:
        print(df_daily.head())
else:
    print(f"'{target_stock_name}' 종목을 찾을 수 없습니다.")


[1] 거래량 상위 주식 목록
=== 상위 주식 데이터 크롤링 시작 (총 40개 목표) ===
 -> 페이지 1 요청 중...
 -> 페이지 2 요청 중...
     종목코드               종목명     현재가    등락률          거래량     거래대금
0  252670  KODEX 200선물인버스2X     706   0.71  591,277,509  4,225억원
1  114800         KODEX 인버스   2,600   0.39   37,354,351    977억원
2  099440                스맥   7,840  14.29   36,369,935  2,741억원
3  125490             한라캐스트  18,330  17.73   26,823,176  4,868억원
4  462330  KODEX 2차전지산업레버리지   1,600  -2.56   25,182,016    406억원

[2] KOSPI 정보
Basic: {'stockEndType': 'index', 'itemCode': 'KOSPI', 'reutersCode': 'KOSPI', 'symbolCode': 'KOSPI', 'stockName': '코스피', 'itemLogoUrl': 'https://ssl.pstatic.net/imgstock/fn/real/logo/index/IndexKOSPI.svg', 'itemLogoPngUrl': 'https://ssl.pstatic.net/imgstock/fn/real/logo/png/index/IndexKOSPI.png', 'closePrice': '4,028.51', 'compareToPreviousClosePrice': '-7.79', 'compareToPreviousPrice': {'code': '5', 'text': '하락', 'name': 'FALLING'}, 'fluctuationsRatio': '-0.19', 'marketStatus': 'CLOSE', 'localTradedA